# Data loading

MSGPACK --> Parquet.

In [1]:
from ship_routing.app.routing import RoutingResult, RoutingLog
from load_tuning_results import (
    load_results_raw,
    load_result_for_key,
    load_results,
    get_journey_params_df,
    get_hyper_params_df,
    get_runtime_df,
    get_elite_df,
)

In [2]:
from pathlib import Path

import pandas as pd
import geopandas as gpd

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
data_files = sorted(Path("../results/").glob("results_2*.msgpack"))
results = load_results(data_files)
len(results)

records:   0%|          | 0/1001 [00:00<?, ?it/s]

1001

In [5]:
# Load individual DataFrames
df_hyper = get_hyper_params_df(results)
df_journey = get_journey_params_df(results)
df_runtime = get_runtime_df(results)
df_elite = get_elite_df(results)

# Merge: params (1-to-1) → runtime (1-to-1) → elite (1-to-many with left join)
df_merged = (
    df_hyper.merge(df_journey, left_index=True, right_index=True, how="inner")
    .merge(df_runtime, left_index=True, right_index=True, how="inner")
    .merge(df_elite, left_index=True, right_index=True, how="left")
)

df_merged

elite:   0%|          | 0/1001 [00:00<?, ?it/s]

,hyper_population_size,hyper_random_seed,hyper_selection_acceptance_rate_warmup,hyper_mutation_width_fraction_warmup,hyper_mutation_displacement_fraction_warmup,hyper_generations,hyper_offspring_size,hyper_crossover_rounds,hyper_selection_quantile,hyper_selection_acceptance_rate,...,journey_speed_knots,journey_time_resolution_hours,runtime,runtime_seconds,n_elite,elite_length_meters,elite_length_relative,elite_cost_absolute,elite_cost_relative,geometry
filename,,,,,,,,,,,,,,,,,,,,,
result:0008:seed2010129258,256,2010129258,0.3,0.99,0.25,1,4,2,0.25,0.00,...,12.0,6.0,0 days 01:27:18,5238.0,0,6.108673e+06,1.002516,7.209229e+12,0.904939,LINESTRING (-11.000000000000002 50.00000000000...
result:0008:seed2010129258,256,2010129258,0.3,0.99,0.25,1,4,2,0.25,0.00,...,12.0,6.0,0 days 01:27:18,5238.0,1,6.108673e+06,1.002516,7.209229e+12,0.904939,LINESTRING (-11.000000000000002 50.00000000000...
result:0012:seed642567293,256,642567293,0.3,0.99,0.25,1,4,2,0.10,0.25,...,12.0,6.0,0 days 01:26:59,5219.0,0,6.544757e+06,1.074083,6.099380e+12,0.873315,LINESTRING (-11.000000000000002 50.00000000000...
result:0012:seed642567293,256,642567293,0.3,0.99,0.25,1,4,2,0.10,0.25,...,12.0,6.0,0 days 01:26:59,5219.0,1,6.544757e+06,1.074083,6.099380e+12,0.873315,LINESTRING (-11.000000000000002 50.00000000000...
result:0013:seed2157407743,128,2157407743,0.3,0.99,0.10,2,4,1,0.10,0.25,...,12.0,6.0,0 days 01:13:46,4426.0,0,6.249119e+06,1.025565,5.228304e+12,0.874525,"LINESTRING (-80.5 29.99999999999999, -79.48248..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
result:1995:seed4150290824,128,4150290824,0.3,0.99,0.10,1,4,1,0.10,0.00,...,10.0,6.0,0 days 01:34:40,5680.0,1,6.104592e+06,1.001846,6.984526e+12,0.865933,"LINESTRING (-80.5 29.99999999999999, -79.65854..."
result:1996:seed1200463295,128,1200463295,0.3,0.99,0.10,1,4,2,0.25,0.25,...,10.0,6.0,0 days 01:32:47,5567.0,0,6.115988e+06,1.003716,5.957632e+12,0.976325,"LINESTRING (-80.5 29.99999999999999, -79.64782..."
result:1996:seed1200463295,128,1200463295,0.3,0.99,0.10,1,4,2,0.25,0.25,...,10.0,6.0,0 days 01:32:47,5567.0,1,6.115988e+06,1.003716,5.957632e+12,0.976325,"LINESTRING (-80.5 29.99999999999999, -79.64782..."


In [6]:
gpd.GeoDataFrame(df_merged).to_parquet("../results/results_prelim.geoparquet")